#### Imports

In [1]:
import os
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType
import pandas as pd
from env import host, username, password

# Exercises

1. Read the case, department, and source data into their own spark dataframes.
2. Let's see how writing to the local disk works in spark:
    - Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json
    - Inspect your folder structure. What do you notice?
3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [2]:
# Generate spark object to initialize a local Spark JVM process
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/06 10:23:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
def get_connection_url(db, user=username, host=host, password=password):
    '''
    This function uses my info from my env file to
    create a connection url to access the Codeup db.
    '''
    return f'mysql+pymysql://{username}:{password}@{host}/{db}'

In [4]:
url = get_connection_url("311_data")

---
### Case

In [5]:
query = """SELECT * FROM cases"""
cases = pd.read_sql(query, url)
cases = spark.createDataFrame(cases)
cases.show(4)

23/07/06 10:25:35 WARN TaskSetManager: Stage 0 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.
23/07/06 10:25:40 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 0 (TID 0): Attempting to kill Python Worker


+----------+----------------+----------------+------------+---------+-------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+
|   case_id|case_opened_date|case_closed_date|SLA_due_date|case_late|num_days_late|case_closed|   dept_division|service_request_type|   SLA_days|case_status|source_id|     request_address|council_district|
+----------+----------------+----------------+------------+---------+-------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+
|1014127332|     1/1/18 0:42|    1/1/18 12:29|9/26/20 0:42|       NO| -998.5087616|        YES|Field Operations|        Stray Animal|      999.0|     Closed| svcCRMLS|2315  EL PASO ST,...|               5|
|1014127333|     1/1/18 0:46|     1/3/18 8:11| 1/5/18 8:30|       NO| -2.012604167|        YES|     Storm Water|Removal Of Obstru...|4.322222222|     Closed| svcCRMSS|2215  GOL

In [6]:
cases.show(2, vertical=True)

23/07/06 10:25:45 WARN TaskSetManager: Stage 1 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616         
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 1/1/18 0:46          
 case_closed_date     | 1/3/18 8:11          
 SLA_due_date         | 1/5/18 8:30          
 case_late            | NO                   
 num_days_late        | -2.0126041

23/07/06 10:25:49 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 1 (TID 1): Attempting to kill Python Worker


In [14]:
# case.write.json('case_json', mode='overwrite')
# case.write.csv('case_csv', mode='overwrite')

23/07/05 13:38:41 WARN TaskSetManager: Stage 5 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.
23/07/05 13:38:48 WARN TaskSetManager: Stage 6 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


In [7]:
cases.schema

StructType([StructField('case_id', LongType(), True), StructField('case_opened_date', StringType(), True), StructField('case_closed_date', StringType(), True), StructField('SLA_due_date', StringType(), True), StructField('case_late', StringType(), True), StructField('num_days_late', DoubleType(), True), StructField('case_closed', StringType(), True), StructField('dept_division', StringType(), True), StructField('service_request_type', StringType(), True), StructField('SLA_days', DoubleType(), True), StructField('case_status', StringType(), True), StructField('source_id', StringType(), True), StructField('request_address', StringType(), True), StructField('council_district', LongType(), True)])

In [8]:
cases = cases.withColumnRenamed('SLA_due_date', 'case_due_date')
cases.show(2, vertical=True)

23/07/06 10:26:09 WARN TaskSetManager: Stage 2 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 case_due_date        | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616         
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 1/1/18 0:46          
 case_closed_date     | 1/3/18 8:11          
 case_due_date        | 1/5/18 8:30          
 case_late            | NO                   
 num_days_late        | -2.0126041

23/07/06 10:26:13 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 2 (TID 2): Attempting to kill Python Worker


In [9]:
cases = cases.withColumnRenamed('SLA_days', 'case_days')
cases.show(2, vertical=True)

23/07/06 10:26:22 WARN TaskSetManager: Stage 3 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 case_due_date        | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616         
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 case_days            | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 1/1/18 0:46          
 case_closed_date     | 1/3/18 8:11          
 case_due_date        | 1/5/18 8:30          
 case_late            | NO                   
 num_days_late        | -2.0126041

23/07/06 10:26:26 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 3 (TID 3): Attempting to kill Python Worker


In [10]:
# demonstrating we only have yes/no in each field
cases.groupby('case_closed', 'case_late').count().show()

23/07/06 10:26:36 WARN TaskSetManager: Stage 4 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


+-----------+---------+------+
|case_closed|case_late| count|
+-----------+---------+------+
|         NO|      YES|  6525|
|        YES|      YES| 87978|
|         NO|       NO| 11585|
|        YES|       NO|735616|
+-----------+---------+------+



In [11]:
#Let's create two new boolean columns
cases = cases.withColumn("case_closed", expr('case_closed == "YES"')).withColumn(
        "case_late", expr('case_late == "YES"'))

#Select just the two columns
cases.select("case_closed", "case_late").show()

23/07/06 10:26:49 WARN TaskSetManager: Stage 7 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


+-----------+---------+
|case_closed|case_late|
+-----------+---------+
|       true|    false|
|       true|    false|
|       true|    false|
|       true|    false|
|       true|     true|
|       true|    false|
|       true|    false|
|       true|    false|
|       true|    false|
|       true|    false|
|       true|    false|
|       true|    false|
|       true|    false|
|       true|    false|
|       true|    false|
|       true|    false|
|       true|    false|
|       true|    false|
|       true|    false|
|       true|    false|
+-----------+---------+
only showing top 20 rows



23/07/06 10:26:53 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 7 (TID 13): Attempting to kill Python Worker


In [12]:
cases.groupBy('council_district').count().show()

23/07/06 10:26:59 WARN TaskSetManager: Stage 8 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


+----------------+------+
|council_district| count|
+----------------+------+
|               0|  3830|
|               7| 72445|
|               6| 74095|
|               9| 40916|
|               5|114609|
|               1|119309|
|              10| 62926|
|               3|102706|
|               8| 42345|
|               2|114745|
|               4| 93778|
+----------------+------+



In [13]:
cases = cases.withColumn('council_district', col('council_district').cast('string'))

In [14]:
#Before
cases.select('case_opened_date', 'case_closed_date', 'case_due_date').show(5)

23/07/06 10:27:16 WARN TaskSetManager: Stage 11 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


+----------------+----------------+-------------+
|case_opened_date|case_closed_date|case_due_date|
+----------------+----------------+-------------+
|     1/1/18 0:42|    1/1/18 12:29| 9/26/20 0:42|
|     1/1/18 0:46|     1/3/18 8:11|  1/5/18 8:30|
|     1/1/18 0:48|     1/2/18 7:57|  1/5/18 8:30|
|     1/1/18 1:29|     1/2/18 8:13| 1/17/18 8:30|
|     1/1/18 1:34|    1/1/18 13:29|  1/1/18 4:34|
+----------------+----------------+-------------+
only showing top 5 rows



23/07/06 10:27:20 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 11 (TID 23): Attempting to kill Python Worker


In [15]:
#After

fmt = "M/d/yy H:mm"

cases = (
    cases.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_closed_date", to_timestamp("case_closed_date", fmt))
    .withColumn("case_due_date", to_timestamp("case_due_date", fmt))
)

cases.select("case_opened_date", "case_closed_date", "case_due_date").show(5)

23/07/06 10:27:30 WARN TaskSetManager: Stage 12 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


+-------------------+-------------------+-------------------+
|   case_opened_date|   case_closed_date|      case_due_date|
+-------------------+-------------------+-------------------+
|2018-01-01 00:42:00|2018-01-01 12:29:00|2020-09-26 00:42:00|
|2018-01-01 00:46:00|2018-01-03 08:11:00|2018-01-05 08:30:00|
|2018-01-01 00:48:00|2018-01-02 07:57:00|2018-01-05 08:30:00|
|2018-01-01 01:29:00|2018-01-02 08:13:00|2018-01-17 08:30:00|
|2018-01-01 01:34:00|2018-01-01 13:29:00|2018-01-01 04:34:00|
+-------------------+-------------------+-------------------+
only showing top 5 rows



23/07/06 10:27:34 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 12 (TID 24): Attempting to kill Python Worker


In [17]:
#After
cases = cases.withColumn('request_address', trim(lower(cases.request_address)))

cases.select("request_address").show(5, 50)

23/07/06 10:27:53 WARN TaskSetManager: Stage 13 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


+-------------------------------------+
|                      request_address|
+-------------------------------------+
| 2315  el paso st, san antonio, 78207|
|  2215  goliad rd, san antonio, 78223|
|102  palfrey st w, san antonio, 78223|
| 114  la garde st, san antonio, 78223|
|734  clearview dr, san antonio, 78228|
+-------------------------------------+
only showing top 5 rows



23/07/06 10:27:57 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 13 (TID 25): Attempting to kill Python Worker


In [18]:
cases = cases.withColumn('num_weeks_late', expr('num_days_late / 7'))

cases.select("num_days_late", "num_weeks_late").show(5)

23/07/06 10:28:06 WARN TaskSetManager: Stage 14 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


+-------------+--------------------+
|num_days_late|      num_weeks_late|
+-------------+--------------------+
| -998.5087616|        -142.6441088|
| -2.012604167|        -0.287514881|
| -3.022337963|-0.43176256614285713|
| -15.01148148| -2.1444973542857144|
|  0.372164352|0.053166335999999995|
+-------------+--------------------+
only showing top 5 rows



23/07/06 10:28:10 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 14 (TID 26): Attempting to kill Python Worker


In [19]:
cases = cases.withColumn('council_district', col('council_district').cast('int'))

# '%03d' means at least 3 digits, pad with 0s
#
# In order to use the format_string function the way we are, we'll need to
# convert council_district back to an integer temporarily, but the final output
# will be a string.
cases = cases.withColumn('council_district',
                  format_string('%03d', col('council_district').cast('int')))

cases.select("council_district").show(5)

23/07/06 10:28:25 WARN TaskSetManager: Stage 15 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


+----------------+
|council_district|
+----------------+
|             005|
|             003|
|             003|
|             003|
|             007|
+----------------+
only showing top 5 rows



23/07/06 10:28:29 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 15 (TID 27): Attempting to kill Python Worker


In [20]:
cases = (
# calculates the number of days between the current timestamp & case_opened_date
    cases.withColumn(
        'case_age', datediff(current_timestamp(), 'case_opened_date')
    )
# Calculates the number of days between the case_closed_date & case_opened_date
    .withColumn(
        'days_to_closed', datediff('case_closed_date', 'case_opened_date')
    )
# since case_closed is boolean, we can check if it's T/F;
#If the case is not closed, it takes the value from the "case_age" column
#Otherwise, it takes the value from the "days_to_closed" column    
    .withColumn(
        'case_lifetime', 
            when(expr('! case_closed'), col('case_age')).otherwise(col('days_to_closed'))

    )
)

In [27]:
# Extract zipcode
cases = cases.withColumn('zipcode', regexp_extract(col('request_address'), r'\d+$', 0))

# Create a `case_lifetime` feature
cases = (
    cases.withColumn('case_age', datediff(to_timestamp(lit('2018-08-08 10:38:00')), 'case_opened_date'))
    .withColumn('days_to_closed', datediff('case_closed_date', 'case_opened_date'))
    .withColumn('case_lifetime', when(
        col('case_closed'), col('days_to_closed')).otherwise(col('case_age')))
    .withColumn('curr_case_age', datediff(current_timestamp(), 'case_opened_date'))
    .drop('case_age', 'days_to_closed')
)

In [28]:
cases.show(2, vertical=True)

23/07/06 10:30:57 WARN TaskSetManager: Stage 20 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616         
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 case_days            | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 005                  
 num_weeks_late       | -142.6441088         
 case_lifetime        | 0                    
 zipcode              | 78207                
 curr_case_age        | 2012                 
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 2018-01-01

23/07/06 10:31:01 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 20 (TID 32): Attempting to kill Python Worker


---
### Dept

In [22]:
query = """SELECT * FROM dept"""
dept = pd.read_sql(query, url)
dept = spark.createDataFrame(dept)
dept.show(4)

+--------------------+--------------------+----------------------+-------------------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+--------------------+--------------------+----------------------+-------------------+
|     311 Call Center|    Customer Service|      Customer Service|                YES|
|               Brush|Solid Waste Manag...|           Solid Waste|                YES|
|     Clean and Green|Parks and Recreation|    Parks & Recreation|                YES|
|Clean and Green N...|Parks and Recreation|    Parks & Recreation|                YES|
+--------------------+--------------------+----------------------+-------------------+
only showing top 4 rows



In [15]:
# dept.write.json('dept_json', mode='overwrite')
# dept.write.csv('dept_csv', mode='overwrite')

In [23]:
dept.schema

StructType([StructField('dept_division', StringType(), True), StructField('dept_name', StringType(), True), StructField('standardized_dept_name', StringType(), True), StructField('dept_subject_to_SLA', StringType(), True)])

---
### Source

In [24]:
query = """SELECT * FROM source"""
source = pd.read_sql(query, url)
source = spark.createDataFrame(source)
source.show(4)

+-----+---------+----------------+
|index|source_id| source_username|
+-----+---------+----------------+
|    0|   100137|Merlene Blodgett|
|    1|   103582|     Carmen Cura|
|    2|   106463| Richard Sanchez|
|    3|   119403|  Betty De Hoyos|
+-----+---------+----------------+
only showing top 4 rows



In [25]:
# source.write.json('source_json', mode='overwrite')
# source.write.csv('source_csv', mode='overwrite')

In [26]:
source.schema

StructType([StructField('index', LongType(), True), StructField('source_id', StringType(), True), StructField('source_username', StringType(), True)])

## 1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [29]:
cases.filter("! case_closed")\
    .select('case_id', 'case_lifetime', 'case_opened_date', 'case_closed_date', 'num_days_late')\
    .sort(desc("num_days_late")).show(5)

23/07/06 10:35:20 WARN TaskSetManager: Stage 21 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


+----------+-------------+-------------------+----------------+-------------+
|   case_id|case_lifetime|   case_opened_date|case_closed_date|num_days_late|
+----------+-------------+-------------------+----------------+-------------+
|1014206419|          190|2018-01-30 16:54:00|            null|          NaN|
|1014558316|           70|2018-05-30 11:35:00|            null|          NaN|
|1014759588|            0|2018-08-08 10:33:00|            null|          NaN|
|1014598231|           57|2018-06-12 16:53:00|            null|          NaN|
|1014397247|          125|2018-04-05 14:59:00|            null|          NaN|
+----------+-------------+-------------------+----------------+-------------+
only showing top 5 rows



In [30]:
cases.filter("! case_closed")\
    .select('case_id', 'case_lifetime', 'curr_case_age', 'case_opened_date', 'case_closed_date', 'num_days_late')\
    .sort(desc("num_days_late")).show(5)

23/07/06 10:35:31 WARN TaskSetManager: Stage 22 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


+----------+-------------+-------------+-------------------+----------------+-------------+
|   case_id|case_lifetime|curr_case_age|   case_opened_date|case_closed_date|num_days_late|
+----------+-------------+-------------+-------------------+----------------+-------------+
|1013762220|          372|         2165|2017-08-01 10:43:00|            null|          NaN|
|1013939287|          302|         2095|2017-10-10 10:57:00|            null|          NaN|
|1014759588|            0|         1793|2018-08-08 10:33:00|            null|          NaN|
|1013940992|          302|         2095|2017-10-10 16:34:00|            null|          NaN|
|1014397247|          125|         1918|2018-04-05 14:59:00|            null|          NaN|
+----------+-------------+-------------+-------------------+----------------+-------------+
only showing top 5 rows



## 2. How many Stray Animal cases are there?

In [31]:
cases.groupby("service_request_type").count().sort(desc("count")).show(truncate=False)

23/07/06 10:37:57 WARN TaskSetManager: Stage 23 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


+---------------------------------+-----+
|service_request_type             |count|
+---------------------------------+-----+
|No Pickup                        |86855|
|Overgrown Yard/Trash             |65895|
|Bandit Signs                     |32910|
|Damaged Cart                     |30338|
|Front Or Side Yard Parking       |28794|
|Stray Animal                     |26760|
|Aggressive Animal(Non-Critical)  |24882|
|Cart Exchange Request            |22024|
|Junk Vehicle On Private Property |21473|
|Pot Hole Repair                  |20616|
|Alley-Way Maintenance            |20214|
|Lost/Stolen Cart                 |18731|
|Right Of Way/Sidewalk Obstruction|17699|
|Dead Animal - Dog                |16714|
|Cart Delivery                    |15471|
|Dead Animal - Cat                |14983|
|Animal Neglect                   |13441|
|Dead Animal - Misc               |13234|
|Trapped/Confined Animal          |11354|
|Public Nuisance(Own Animal)      |10715|
+---------------------------------

In [33]:
cases.filter(cases.service_request_type == lit("Stray Animal")).count()

23/07/06 10:38:19 WARN TaskSetManager: Stage 26 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


26760

## 3. How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [35]:
cases.filter(cases.dept_division == "Field Operations").filter(cases.service_request_type != "Officer Standby").count()

23/07/06 10:39:07 WARN TaskSetManager: Stage 29 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


113902

## 4. Convert the council_district column to a string column.

In [36]:
cases = cases.withColumn('council_district', col('council_district').cast('string'))

## 5. Extract the year from the case_closed_date column.

In [37]:
cases.withColumn("year", year("case_closed_date")).select("service_request_type", 'case_closed_date', 'year')\
.show(5)

23/07/06 10:39:45 WARN TaskSetManager: Stage 32 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+-------------------+----+
|service_request_type|   case_closed_date|year|
+--------------------+-------------------+----+
|        Stray Animal|2018-01-01 12:29:00|2018|
|Removal Of Obstru...|2018-01-03 08:11:00|2018|
|Removal Of Obstru...|2018-01-02 07:57:00|2018|
|Front Or Side Yar...|2018-01-02 08:13:00|2018|
|Animal Cruelty(Cr...|2018-01-01 13:29:00|2018|
+--------------------+-------------------+----+
only showing top 5 rows



23/07/06 10:39:49 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 32 (TID 76): Attempting to kill Python Worker


## 6. Convert num_days_late from days to hours in new columns num_hours_late.

In [39]:
cases.withColumn("num_hours_late", round(expr("num_days_late * 24"), 1))\
.select("num_days_late", 'num_hours_late').show(5)

23/07/06 10:40:07 WARN TaskSetManager: Stage 33 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


+-------------+--------------+
|num_days_late|num_hours_late|
+-------------+--------------+
| -998.5087616|      -23964.2|
| -2.012604167|         -48.3|
| -3.022337963|         -72.5|
| -15.01148148|        -360.3|
|  0.372164352|           8.9|
+-------------+--------------+
only showing top 5 rows



23/07/06 10:40:11 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 33 (TID 77): Attempting to kill Python Worker


## 7. Join the case data with the source and department data.

In [40]:
df = cases.join(source, "source_id", "left").drop(source.source_id)

df.show(2, vertical=True)

23/07/06 10:40:59 WARN TaskSetManager: Stage 34 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.
23/07/06 10:41:04 WARN PythonRunner: Detected deadlock while completing task 3.0 in stage 34 (TID 81): Attempting to kill Python Worker
23/07/06 10:41:04 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 34 (TID 78): Attempting to kill Python Worker
23/07/06 10:41:04 WARN PythonRunner: Detected deadlock while completing task 1.0 in stage 34 (TID 79): Attempting to kill Python Worker
23/07/06 10:41:04 WARN PythonRunner: Detected deadlock while completing task 4.0 in stage 34 (TID 82): Attempting to kill Python Worker
23/07/06 10:41:04 WARN PythonRunner: Detected deadlock while completing task 6.0 in stage 34 (TID 84): Attempting to kill Python Worker
23/07/06 10:41:04 WARN PythonRunner: Detected deadlock while completing task 2.0 in stage 34 (TID 80): Attempting to kill Python Worker
23/07/06 10:41:04 WARN PythonRunner: Dete

-RECORD 0------------------------------------
 source_id            | 119868               
 case_id              | 1014744357           
 case_opened_date     | 2018-08-02 16:53:00  
 case_closed_date     | 2018-08-03 14:33:00  
 case_due_date        | 2018-08-09 16:53:00  
 case_late            | false                
 num_days_late        | -6.096967593         
 case_closed          | true                 
 dept_division        | Waste Collection     
 service_request_type | Damaged Cart         
 case_days            | 7.0                  
 case_status          | Closed               
 request_address      | 555  gettysburg r... 
 council_district     | 007                  
 num_weeks_late       | -0.8709953704285713  
 case_lifetime        | 1                    
 zipcode              | 78228                
 curr_case_age        | 1799                 
 index                | 5                    
 source_username      | Michelle San Miguel  
-RECORD 1-------------------------

In [41]:
df = df.join(dept, "dept_division", "left")\
    .drop(dept.dept_division)\
    .drop(dept.dept_name)\
    .withColumnRenamed('standardized_dept_name', 'department')\
    .withColumn('dept_subject_to_SLA', col('dept_subject_to_SLA') == 'YES')
df.show(2, vertical=True)

23/07/06 10:41:10 WARN TaskSetManager: Stage 39 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.
23/07/06 10:41:14 WARN PythonRunner: Detected deadlock while completing task 7.0 in stage 39 (TID 102): Attempting to kill Python Worker
23/07/06 10:41:14 WARN PythonRunner: Detected deadlock while completing task 3.0 in stage 39 (TID 98): Attempting to kill Python Worker
23/07/06 10:41:14 WARN PythonRunner: Detected deadlock while completing task 1.0 in stage 39 (TID 96): Attempting to kill Python Worker
23/07/06 10:41:14 WARN PythonRunner: Detected deadlock while completing task 4.0 in stage 39 (TID 99): Attempting to kill Python Worker
23/07/06 10:41:14 WARN PythonRunner: Detected deadlock while completing task 6.0 in stage 39 (TID 101): Attempting to kill Python Worker
23/07/06 10:41:14 WARN PythonRunner: Detected deadlock while completing task 2.0 in stage 39 (TID 97): Attempting to kill Python Worker
23/07/06 10:41:14 WARN PythonRunner: De

-RECORD 0------------------------------------
 dept_division        | Waste Collection     
 source_id            | 119868               
 case_id              | 1014744357           
 case_opened_date     | 2018-08-02 16:53:00  
 case_closed_date     | 2018-08-03 14:33:00  
 case_due_date        | 2018-08-09 16:53:00  
 case_late            | false                
 num_days_late        | -6.096967593         
 case_closed          | true                 
 service_request_type | Damaged Cart         
 case_days            | 7.0                  
 case_status          | Closed               
 request_address      | 555  gettysburg r... 
 council_district     | 007                  
 num_weeks_late       | -0.8709953704285713  
 case_lifetime        | 1                    
 zipcode              | 78228                
 curr_case_age        | 1799                 
 index                | 5                    
 source_username      | Michelle San Miguel  
 department           | Solid Wast

## 8. Are there any cases that do not have a request source?

In [42]:
df.filter(df.source_id == "null").count()

23/07/06 10:41:56 WARN TaskSetManager: Stage 47 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


0

In [43]:
df.filter(df.source_id == "").count()

23/07/06 10:41:59 WARN TaskSetManager: Stage 52 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


0

## 9. What are the top 10 service request types in terms of number of requests?

In [44]:
df.groupby("service_request_type").count().sort(desc("count")).show(10, 50)

23/07/06 10:42:11 WARN TaskSetManager: Stage 57 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


+--------------------------------+-----+
|            service_request_type|count|
+--------------------------------+-----+
|                       No Pickup|89210|
|            Overgrown Yard/Trash|66403|
|                    Bandit Signs|32968|
|                    Damaged Cart|31163|
|      Front Or Side Yard Parking|28920|
|                    Stray Animal|27361|
| Aggressive Animal(Non-Critical)|25492|
|           Cart Exchange Request|22608|
|Junk Vehicle On Private Property|21649|
|                 Pot Hole Repair|20827|
+--------------------------------+-----+
only showing top 10 rows



## 10. What are the top 10 service request types in terms of average days late?

In [45]:
df.groupby("service_request_type").agg(round(mean("num_days_late"), 1).alias("avg_late"))\
.sort(desc("avg_late")).show(10, 50)

23/07/06 10:43:10 WARN TaskSetManager: Stage 69 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


+--------------------------------------+--------+
|                  service_request_type|avg_late|
+--------------------------------------+--------+
|      Request for Research/Information|     NaN|
|CCO_Request for Research/Information_1|     NaN|
|                    Zoning: Junk Yards|   176.0|
|            Labeling for Used Mattress|   162.4|
|     Record Keeping of Used Mattresses|   154.0|
|Signage Requied for Sale of Used Mattr|   151.6|
|              Storage of Used Mattress|   142.1|
|                  Zoning: Recycle Yard|   135.9|
|        Donation Container Enforcement|   131.8|
|   License Requied Used Mattress Sales|   128.8|
+--------------------------------------+--------+
only showing top 10 rows



## 11. Does number of days late depend on department?

In [46]:
df.groupby("department").agg(round(mean("num_days_late"), 1). alias("avg_late"))\
.sort(desc("avg_late")).show(10, 50)

23/07/06 10:43:17 WARN TaskSetManager: Stage 81 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


+------------------------+--------+
|              department|avg_late|
+------------------------+--------+
|            City Council|     NaN|
|        Customer Service|    59.7|
|             Solid Waste|    -2.2|
|            Metro Health|    -4.9|
|      Parks & Recreation|    -5.3|
|Trans & Cap Improvements|   -20.6|
|    DSD/Code Enforcement|   -38.4|
|    Animal Care Services|  -226.5|
+------------------------+--------+



## 12. How do number of days late depend on department and request type?

In [47]:
df.groupby("department", "service_request_type")\
.agg(round(avg("num_days_late"), 1).alias("avg_days_late")).sort("department").show(50,50)

23/07/06 10:43:21 WARN TaskSetManager: Stage 93 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+----------------------------------------+-------------+
|          department|                    service_request_type|avg_days_late|
+--------------------+----------------------------------------+-------------+
|Animal Care Services|                Injured Animal(Critical)|         -0.1|
|Animal Care Services|             Aggressive Animal(Critical)|         16.7|
|Animal Care Services|                   Animal Bite(Critical)|          0.0|
|Animal Care Services|                 Trapped/Confined Animal|          0.2|
|Animal Care Services|             City Council Animal Request|         -1.5|
|Animal Care Services|                            Stray Animal|       -998.8|
|Animal Care Services|                Animal Cruelty(Critical)|         -0.1|
|Animal Care Services|             Public Nuisance(Own Animal)|         -2.2|
|Animal Care Services|                          Animal Neglect|          7.3|
|Animal Care Services|                         Officer Standby| 